In [1]:
import sys
import os

from portopt.portfolio import Portfolio
from portopt.config import load_config
from portopt.market_data import get_tickers_data
from portopt.porteval import simulate_model_portfolio
from portopt.utils import plot_time_series

In [ ]:
# required file paths
portfolio_dir = "../data/portfolio"
config_file = os.path.join(portfolio_dir, "config.yml")
holdings_files = os.path.join(portfolio_dir, "holdings")
factor_weights_file = os.path.join(portfolio_dir, "asset_class_weights_matrix.csv")

print("portfolio_dir:", portfolio_dir)
print("config_file:", config_file)
print("factor_weights_file:", factor_weights_file)
print("holdings_files:", holdings_files)

In [ ]:
# load config
config = load_config(config_file)
import pprint
pprint.pprint(config)

In [ ]:
# create poprtfolio object
portfolio = Portfolio(config, factor_weights_file, holdings_files)
portfolio

In [ ]:
# convert portfolio to model portfolio where a model portfolio
# is a dictionary mapping tickers to weights
model_portfolio = portfolio.getMetrics('Ticker')['Allocation'].to_dict()
model_portfolio

In [ ]:
# filter out options and money market funds from model portfolio
# (can't get historical data for these???)
from portopt.market_data import is_option_ticker
from portopt.market_data import is_money_market_ticker
model_portfolio = {k: v for k, v in model_portfolio.items() if not is_option_ticker(k) and not is_money_market_ticker(k)}

# remove crypto assets from model portfolio
# (not enough historical data for crypto asset classes)
model_portfolio = {k: v for k, v in model_portfolio.items() if k not in ["FBTC", "FETH"]}

model_portfolio


In [ ]:
# start date for backtest
start_date = "2000-01-01"

# get historical price data for model portfolio
# (incorporate this into the portfolio object's getPrices() method?)
historical_prices, stats = get_tickers_data(list(model_portfolio.keys()),
                              start_date=start_date,
                              end_date=None,
                              price_type="Adj Close",
                              verbose=True)
historical_prices

In [ ]:
# simulate model portfolio
simulated_portfolio = simulate_model_portfolio(model_portfolio,
                         start_date=start_date,
                         end_date=None,
                         rebalance_freq="quarterly",
                         price_data=historical_prices,
                         engine="bt",
                         verbose=True)

plot_time_series(simulated_portfolio.prices)
print(simulated_portfolio.prices.shape)